In [1]:
from IPython.display import display, Math
from IPython.display import display as disp

import numpy as np
import matplotlib.pyplot as plt

# Transimissão óptica ASK(PAM) ideal

<img src="../figures/sinalgenerico.png" width="500">

A ideia é excitar o modulador óptico (MZM) com sinais elétricos gerados a partir de uma sequência binária:

$$ Bits (bits) \rightarrow Símbolos (symbTx) \rightarrow Pulsos (pulseTx) \rightarrow Sinal Elétrico (sigTxe) \rightarrow Sinal Óptico (sigTxo) $$

In [11]:
from commpy.utilities  import signal_power, upsample
from utils.dsp import pulseShape, firFilter
from utils.models import mzm

In [27]:
# Parâmetros da simulação:
SpS = 32            # Amostras por Símbolo
Rs  = 10e9          # Taxa de símbolos                  [bauds]
Ts  = 1/Rs          # Período de símbolo em segundos    [segundos]
Fa  = 1/(Ts/SpS)    # Frequência de amostragem do sinal (amostras/segundo)
Ta  = 1/Fa          # Período de amostragem

# Par#ametros do Modulador Óptico p/ transmitir em potência:
P0 = 100e-3         # potência da portadora CW na entrada no MZM
Vπ = 2
Vb = -Vπ/2
Ai = np.sqrt(P0)


In [1]:
# Gera sequência binária:
bits = np.random.randint(2, size=20)    

# Mapeia bits em Símbolos:
ASK    = np.array([-1,1])                  # Regra de modulação (Utiliza-se somente amplitude)
symbTx = ASK[bits]                         # f{[0],[1]} -> {[-1],[1]}

# Formatando pulso retangular ideal:
symbTx_ups = upsample(symbTx, SpS).real    # Adiciona SpS ao trem de pulsos
pulse  = pulseShape('rect', SpS)           # Gera pulso retangular (ideal)
pulse  = pulse/max(abs(pulse))
sigTxe = firFilter(pulse, symbTx_ups).real # Convolução símbolos*pulso

# Passando o sinal Elétrico pelo MZM:
sigTxo = mzm(Ai, Vπ, sigTxe, Vb)           # Sinal óptico 
t = (0.5 + np.arange(0, symbTx_ups.size))*(Ts/1e-12)


# Plotando Resultados:
symbTx_ups[symbTx_ups==0] = np.nan
plt.figure()
plt.plot(t, sigTxe, '-', linewidth=3)
plt.plot(t, symbTx_ups, 'o')
plt.xlabel('tempo [ps]')
plt.ylabel('amplitude [V]')
plt.title('$\sum_{n}\;s_{n}\;p(t-n T_s)$')
plt.grid()

plt.figure()
plt.plot(t, sigTxo, '-', linewidth=3)
plt.xlabel('tempo [ps]')
plt.ylabel('Potência [W]')
plt.title('$\sqrt{P_0}\sum_{n}\;b_{n}\;p(t-n T_s)$')
plt.grid()


NameError: name 'np' is not defined